In [106]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier,DecisionTreeRegressor,  plot_tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,  f1_score, r2_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from tqdm import tqdm 
from sklearn.ensemble import RandomForestClassifier , StackingClassifier
from sklearn.compose import make_column_transformer, make_column_selector
import numpy as np
from sklearn.impute import SimpleImputer
from xgboost import XGBRegressor, XGBClassifier
from lightgbm import LGBMRegressor , LGBMClassifier
from catboost import CatBoostRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB

In [85]:
df = pd.read_csv('BreastCancer.csv')
df

Code  Clump  UniCell_Size  Uni_CellShape  MargAdh  SEpith  BareN  \
0       61634      5             4              3        1       2      2   
1       63375      9             1              2        6       4     10   
2       76389     10             4              7        2       2      8   
3       95719      6            10             10       10       8     10   
4      128059      1             1              1        1       2      5   
..        ...    ...           ...            ...      ...     ...    ...   
694   1369821     10            10             10       10       5     10   
695   1371026      5            10             10       10       4     10   
696   1371920      5             1              1        1       2      1   
697   8233704      4             1              1        1       1      1   
698  13454352      1             1              3        1       2      1   

     BChromatin  NoemN  Mitoses      Class  
0             2      3        1     Benign  
1             7      7        2  Malignant  
2             6      1        1  Malignant  
3             7     10        7  Malignant  
4             5      1        1     Benign  
..          ...    ...      ...        ...  
694          10     10        7  Malignant  
695           5      6        3  Malignant  
696           3      2        1     Benign  
697           2      1        1     Benign  
698           2      1        1     Benign  

[699 rows x 11 columns]

In [86]:
X , y = df.drop(['Class','Code'], axis=1) , df['Class']
X_train, X_test, Y_train, Y_test = train_test_split(X,y,test_size=0.3, random_state=24)
X_train.dtypes

Clump            int64
UniCell_Size     int64
Uni_CellShape    int64
MargAdh          int64
SEpith           int64
BareN            int64
BChromatin       int64
NoemN            int64
Mitoses          int64
dtype: object

In [87]:
svm = SVC(kernel='linear')
lr = LogisticRegression()
dtc = DecisionTreeClassifier(random_state=24)
rf = RandomForestClassifier(random_state= 24 , n_estimators = 25)

stack =  StackingClassifier( estimators= [('SVM', svm) , ('LR', lr) , ('DTC' , dtc)], final_estimator= rf)

stack.fit(X_train , Y_train )
y_pred =  stack.predict(X_test)

print(f1_score(Y_test , y_pred , pos_label='Malignant'))

0.9436619718309859


In [88]:
#Enabling Passthrough 
svm = SVC(kernel='linear')
lr = LogisticRegression()
dtc = DecisionTreeClassifier(random_state=24)
rf = RandomForestClassifier(random_state= 24 , n_estimators = 25)

stack =  StackingClassifier( estimators= [('SVM', svm) , ('LR', lr) , ('DTC' , dtc)], final_estimator= rf, passthrough = True)

stack.fit(X_train , Y_train )
y_pred =  stack.predict(X_test)

print(f1_score(Y_test , y_pred , pos_label='Malignant'))

0.958904109589041


## With HR Dataset

In [98]:
from sklearn.pipeline import Pipeline

df = pd.read_csv('HR_comma_sep.csv')

In [91]:
df

satisfaction_level  last_evaluation  number_project  \
0                    0.38             0.53               2   
1                    0.80             0.86               5   
2                    0.10             0.77               6   
3                    0.92             0.85               5   
4                    0.89             1.00               5   
...                   ...              ...             ...   
14990                0.40             0.57               2   
14991                0.37             0.48               2   
14992                0.37             0.53               2   
14993                0.11             0.96               6   
14994                0.37             0.52               2   

       average_montly_hours  time_spend_company  Work_accident  left  \
0                       157                   3              0     1   
1                       262                   6              0     1   
2                       247                   4              0     1   
3                       259                   5              0     1   
4                       224                   5              0     1   
...                     ...                 ...            ...   ...   
14990                   151                   3              0     1   
14991                   160                   3              0     1   
14992                   143                   3              0     1   
14993                   280                   4              0     1   
14994                   158                   3              0     1   

       promotion_last_5years Department  salary  
0                          0      sales     low  
1                          0      sales  medium  
2                          0      sales     low  
3                          0      sales     low  
4                          0      sales     low  
...                      ...        ...     ...  
14990                      0    support     low  
14991                      0    support     low  
14992                      0    support     low  
14993                      0    support     low  
14994                      0    support     low  

[14995 rows x 10 columns]

In [92]:
X,y = df.drop('left', axis = 1), df['left']

In [93]:
X_train, X_test, Y_train, Y_test = train_test_split(X,y,test_size=0.3, random_state=24)
X_train.dtypes

satisfaction_level       float64
last_evaluation          float64
number_project             int64
average_montly_hours       int64
time_spend_company         int64
Work_accident              int64
promotion_last_5years      int64
Department                object
salary                    object
dtype: object

In [100]:
X_ohe = OneHotEncoder(sparse_output=False,handle_unknown='ignore', drop='first').set_output(transform='pandas')
ct = make_column_transformer((X_ohe,make_column_selector(dtype_include=object)),
                            ('passthrough',make_column_selector(dtype_exclude=object)),
                            verbose_feature_names_out=False).set_output(transform='pandas')

X_trn = ct.fit_transform(X_train)
X_tst = ct.transform(X_test)

In [95]:
lr = LogisticRegression()
lda = LinearDiscriminantAnalysis()
gb = GaussianNB()
dtc = DecisionTreeClassifier()

stack = StackingClassifier(estimators=[('lr', lr),('lda', lda), ('GB', gb)] , final_estimator=dtc, passthrough=True)
stack.fit(X_trn,Y_train)
y_pred = stack.predict(X_tst)
print(f1_score(Y_test,y_pred,pos_label=1))


C:\Users\dbda.STUDENTSDC\AppData\Local\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\dbda.STUDENTSDC\AppData\Local\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules

0.9455750118315192


In [112]:

lgbm = LGBMClassifier(random_state=24)
dtc = DecisionTreeClassifier()

pt = [False, True]
finals = [lgbm , dtc]
scores = []

for f in finals:
    for p in pt: 
        stack = StackingClassifier(estimators=[('lr', lr),('lda', lda), ('GB', gb)] , 
                                   final_estimator=f, passthrough=p)
        pipe = Pipeline([('CT' ,ct) , ('Stack', stack)])
        pipe.fit(X_trn, Y_train)
        Y_pred = pipe.predict(X_tst)
        scores.append([f , p , f1_score(Y_test , y_pred)])

df1 = pd.DataFrame(scores , columns = ['F' , 'P' , 'F1'])
df1.sort_values('F1', ascending=False)
        


C:\Users\dbda.STUDENTSDC\AppData\Local\anaconda3\Lib\site-packages\sklearn\utils\_tags.py:354: FutureWarning: The LGBMClassifier or classes from which it inherits use `_get_tags` and `_more_tags`. Please define the `__sklearn_tags__` method, or inherit from `sklearn.base.BaseEstimator` and/or other appropriate mixins such as `sklearn.base.TransformerMixin`, `sklearn.base.ClassifierMixin`, `sklearn.base.RegressorMixin`, and `sklearn.base.OutlierMixin`. From scikit-learn 1.7, not defining `__sklearn_tags__` will raise an error.
  warnings.warn(
C:\Users\dbda.STUDENTSDC\AppData\Local\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/sta

[LightGBM] [Info] Number of positive: 2513, number of negative: 7983
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000062 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 765
[LightGBM] [Info] Number of data points in the train set: 10496, number of used features: 3
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.239425 -> initscore=-1.155837
[LightGBM] [Info] Start training from score -1.155837


C:\Users\dbda.STUDENTSDC\AppData\Local\anaconda3\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
C:\Users\dbda.STUDENTSDC\AppData\Local\anaconda3\Lib\site-packages\sklearn\utils\_tags.py:354: FutureWarning: The LGBMClassifier or classes from which it inherits use `_get_tags` and `_more_tags`. Please define the `__sklearn_tags__` method, or inherit from `sklearn.base.BaseEstimator` and/or other appropriate mixins such as `sklearn.base.TransformerMixin`, `sklearn.base.ClassifierMixin`, `sklearn.base.RegressorMixin`, and `sklearn.base.OutlierMixin`. From scikit-learn 1.7, not defining `__sklearn_tags__` will raise an error.
  warnings.warn(
C:\Users\dbda.STUDENTSDC\AppData\Local\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number 

[LightGBM] [Info] Number of positive: 2513, number of negative: 7983
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000594 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1182
[LightGBM] [Info] Number of data points in the train set: 10496, number of used features: 21
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.239425 -> initscore=-1.155837
[LightGBM] [Info] Start training from score -1.155837


C:\Users\dbda.STUDENTSDC\AppData\Local\anaconda3\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
C:\Users\dbda.STUDENTSDC\AppData\Local\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\dbda.STUDENTSDC\AppData\Local\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iteratio

F      P        F1
0  LGBMClassifier(random_state=24)  False  0.945575
1  LGBMClassifier(random_state=24)   True  0.945575
2         DecisionTreeClassifier()  False  0.945575
3         DecisionTreeClassifier()   True  0.945575